# Data Generation

In [1]:
from datetime import datetime, timedelta
import json
import random

In [2]:
trades = ['framing carpenter','electrician','plumber','finishing carpenter', 'painter', 'paver']
today = datetime.utcnow()

In [3]:
def number_of_rooms(max_rooms=6):
    length = random.randrange(3, max_rooms, 1)
    return range(0,length)

In [4]:
def moc_deadlines(trades=[]):
    return [
    (datetime.utcnow() + timedelta(days=index)).strftime('%Y-%m-%dT%H:%M:%SZ') 
     for index,trade in enumerate(trades)
    ]

In [5]:
def moc_room_pipeline(trades=[], deadlines=[]):
    return [
        {'tradesmen': trade, 'deadliine': deadline} for trade, deadline in zip(trades,deadlines)
    ] 
        

In [6]:
def subset(trades=[]):
    start = random.randrange(0, int(len(trades)*0.75)+1, 1)
    return trades[start:]

In [7]:
building_data = {}

for floor in range(4):
    room_data = {}
    for room in number_of_rooms():
        trade_subset = subset(trades)
        pipeline = moc_room_pipeline(trade_subset, moc_deadlines(trade_subset))
        room_data[f'room_{floor}{room}'] = pipeline
    building_data[f'floor_{floor}'] = room_data

In [8]:
with open('roomData.json', 'w') as json_file:
    json.dump(building_data, json_file)